In [1]:
import numpy as np
import datacube
from datacube.utils import geometry
import odc.ui
from odc.ui import to_png_data
from odc.ui import mk_data_uri
from odc.ui import to_jpeg_data
from odc.algo import to_rgba, is_rgb
import xarray as xr
import cv2
import geopandas as gpd
from typing import Tuple
from typing import Union
from typing import Optional
from PIL import Image
from PIL import ImageEnhance
from shapely.geometry.polygon import Polygon
from shapely.geometry import LineString, MultiPolygon
from shapely.ops import polygonize, unary_union
import leafmap
from ipyleaflet import Map, basemaps
from ipywidgets import widgets as w
import sys 
print(sys.path)
import json
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from odc.algo import xr_geomedian
from pyproj import Proj, transform

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


['/home/jovyan/cropmapping_open_datacube_dea/Vision model', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/jovyan/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/home/jovyan/.ipython']


In [2]:
dc = datacube.Datacube(app='farm_plot_delineation')

In [3]:
def apply_function_over_custom_times(ds, func, func_name, time_ranges):
    output_list = []

    for timelabel, timeslice in time_ranges.items():

        if isinstance(timeslice, slice):
            ds_timeslice = ds.sel(time=timeslice)
        else:
            ds_timeslice = ds.sel(time=timeslice, method="nearest")

        ds_modified = func(ds_timeslice)

        rename_dict = {
            key: f"{key}_{func_name}_{timelabel}" for key in list(ds_modified.keys())
        }

        ds_modified = ds_modified.rename(name_dict=rename_dict)

        if "time" in list(ds_modified.coords):
            ds_modified = ds_modified.reset_coords().drop_vars(["time", "spatial_ref"])

        output_list.append(ds_modified)

    return output_list


def geomedian_with_indices_wrapper(ds):
    indices = ["NDVI", "SAVI", "NDMI"]
    satellite_mission = "s2"

    ds_geomedian = xr_geomedian(ds)

    ds_geomedian = calculate_indices(
        ds_geomedian,
        index=indices,
        drop=False,
        satellite_mission=satellite_mission)

    return ds_geomedian

def get_years(ds):
    return np.unique(ds.time.dt.year.values)

def get_months(ds):
    return np.unique(ds.time.dt.month.values)

def cus1_gm_1(query):
    # Connnect to datacube
    dc = datacube.Datacube(app="crop_type_ml")
    
    ds = load_ard(
        dc=dc,
        products=["s2_l2a"],
        verbose=False,
        **query)
    
    time_ranges = {
        "gm": slice(f"{get_years(ds)[0]}-{get_months(ds)[0]}-01", f"{get_years(ds)[0]}-{get_months(ds)[0]}-31")}

    # Apply geomedian over time ranges and calculate band indices
    s2_geomad_list = apply_function_over_custom_times(ds, geomedian_with_indices_wrapper, "s2", time_ranges)
    ds_list = []
    ds_list.extend(s2_geomad_list)
    ds_final = xr.merge(ds_list)

    return ds_final

In [4]:
def xr_bounds(x, crs=None) -> Tuple[Tuple[float, float], Tuple[float, float]]:

    def get_range(a: np.ndarray) -> Tuple[float, float]:
        b = (a[1] - a[0]) * 0.5
        return a[0] - b, a[-1] + b

    if "latitude" in x.coords:
        r1, r2 = (get_range(a.values) for a in (x.latitude, x.longitude))
        p1, p2 = ((r1[i], r2[i]) for i in (0, 1))
        return p1, p2

    if crs is None:
        geobox = getattr(x, "geobox", None)
        if geobox:
            crs = geobox.crs

    if crs is None:
        raise ValueError("Need to supply CRS or use latitude/longitude coords")

    if not all(d in x.coords for d in crs.dimensions):
        raise ValueError("Incompatible CRS supplied")

    (t, b), (l, r) = (get_range(x.coords[dim].values) for dim in crs.dimensions)

    l, b, r, t = box(l, b, r, t, crs).to_crs(epsg4326).boundingbox
    return ((t, r), (b, l))



def mk_image_overlay(
    xx: Union[xr.Dataset, xr.DataArray],
    clamp: Optional[float] = None,
    bands: Optional[Tuple[str, str, str]] = None,
    layer_name="Image",
    fmt="png",
    **opts
):
    """Create ipyleaflet.ImageLayer from raster data.
    xx - xarray.Dataset that will be converted to RGBA or
         xarray.DataArray that is already in RGB(A) format
    clamp, bands -- passed on to to_rgba(..), only used when xx is xarray.Dataset
    Returns
    =======
    ipyleaflet.ImageOverlay or a list of them one per time slice
    """

    comp, mime = dict(
        png=(to_png_data, "image/png"),
        jpg=(to_jpeg_data, "image/jpeg"),
        jpeg=(to_jpeg_data, "image/jpeg"),
    ).get(fmt.lower(), (None, None))

    if comp is None or mime is None:
        raise ValueError("Only support png an jpeg formats")

    if "time" in xx.dims:
        nt = xx.time.shape[0]
        if nt == 1:
            xx = xx.isel(time=0)
        else:
            return [
                mk_image_overlay(
                    xx.isel(time=t),
                    clamp=clamp,
                    bands=bands,
                    layer_name="{}-{}".format(layer_name, t),
                    fmt=fmt,
                    **opts
                )
                for t in range(nt)
            ]

    if isinstance(xx, xr.Dataset):
        rgba = to_rgba(xx, clamp=clamp, bands=bands)
    else:
        if not is_rgb(xx):
            raise ValueError("Expect RGB xr.DataArray")
        rgba = xx

    im_url = mk_data_uri(comp(rgba.values, **opts), mime)
    return im_url, rgba


def convert_list(list_):
    converted_list = []
    for i in range(len(list_)):
        converted_list.append( list(transform(inProj,outProj,ds.x.values[list_[i][0]],ds.y.values[list_[i][1]])) )
    return converted_list

def simplify_crop(polygon):
    lineString = LineString(polygon)
    lr = LineString(lineString.coords[:] + lineString.coords[0:1])
    mls = unary_union(lr)
    mp = MultiPolygon(list(polygonize(mls)))
    mp = mp.buffer(0)
    if mp.geom_type == 'MultiPolygon':
        new_polygon = mp[0].exterior.coords[:]
    elif mp.geom_type == 'Polygon':
        new_polygon = mp.exterior.coords[:]
    new_polygon = [(int(x), int(y)) for (x, y) in new_polygon]
    return new_polygon


def enhance_image(img , contrast=1, sharpness=1, brightness=1):
    enhanced_img = Image.fromarray(img)
    enhanced_img = ImageEnhance.Contrast(enhanced_img).enhance(contrast)
    enhanced_img = ImageEnhance.Sharpness(enhanced_img).enhance(sharpness)
    enhanced_img = ImageEnhance.Brightness(enhanced_img).enhance(brightness)
    enhanced_img = np.asarray(enhanced_img)
    return enhanced_img

def equalize_histogram(img):
    img_new = img.copy()
    img_new[:,:,0] = cv2.equalizeHist(img[:,:,0])
    img_new[:,:,1] = cv2.equalizeHist(img[:,:,1])
    img_new[:,:,2] = cv2.equalizeHist(img[:,:,2])
    return img_new

def find_edges(img):
    if config_dict["thresholding_type"] == "sigma":
        v_med = np.median(img)
        lower = self.config["sigma"] + v_med
        upper = lower*3  
    elif config_dict["thresholding_type"] == "fixed":
        lower, upper = config_dict["thresholds"]  
    edges = cv2.Canny(img, lower, upper)   
    if config_dict["save_intermediate_steps"]:
        cv2.imwrite("zone_delimitation_edges.png", edges)       
    return edges

def structure_edges(edges):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, config_dict["morph_kernel"])
    structured_edges = cv2.morphologyEx(edges, config_dict["morphism_type"], kernel)
    if config_dict["save_intermediate_steps"]:
        cv2.imwrite("zone_delimitation_structured_edges.png", structured_edges)
    return structured_edges

def find_contours(structured_edges, rbg_img):
    contours, hierarchy = cv2.findContours(structured_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if config_dict["save_intermediate_steps"]:
        img_with_contours = np.copy(rbg_img) 
        cv2.drawContours(img_with_contours, contours, -1, (0,255,0), 1)
        cv2.imwrite("zone_delineation_picture_contours.png", img_with_contours)
    return contours

def approximate_contours(contours):
    approximations = []
    for contour in contours: 
        if config_dict["approximation_type"] == "classic":
            perimeter = cv2.arcLength(contour, True)
            # Higer simplicity coef means polygons are pushed to have less points
            simplicity_coef = config_dict["polygon_simplicity_coef"]
            approximation = cv2.approxPolyDP(contour, simplicity_coef * perimeter, True) 
        elif config_dict["approximation_type"] == "hull":
            approximation = cv2.convexHull(contour) 
        if len(approximation) >= 3:
            approximations.append(approximation) 
    return approximations

def extract_polygons(approximations):
    polygons = []
    for approximation in approximations:
        polygon = [(edge[0,0], edge[0,1]) for edge in approximation] 
        polygon = simplify_crop(polygon)
        polygons.append(polygon) 
    return polygons

In [5]:
gdf = gpd.read_file('/home/jovyan/cropmapping_open_datacube_dea/tadla_test.geojson')
geom = geometry.Geometry(gdf.iloc[0].geometry.__geo_interface__,geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))

In [6]:
query = {'time': ('2019-03'),
         'measurements': ['blue',
                         'green',
                         'red',
                         'nir',
                         'swir_1',
                         'swir_2',
                         'red_edge_1',
                         'red_edge_2',
                         'red_edge_3'],
         'resolution': (-10,10),
         'output_crs': 'EPSG:6933',
         'geopolygon':geom}

config_dict = {'thresholding_type':'fixed', 
               'save_intermediate_steps':True, 
               'approximation_type':'hull', 
               'polygon_simplicity_coef':2,
               'sigma':2,
               'thresholds':[100,200],
               'morph_kernel':(5, 5),
               'morphism_type': cv2.MORPH_CLOSE}

#### Apply field delineation process

In [7]:
ds = cus1_gm_1(query)

In [8]:
ds

<xarray.Dataset>
Dimensions:           (y: 659, x: 779)
Coordinates:
  * y                 (y) float64 3.911e+06 3.911e+06 ... 3.904e+06 3.904e+06
  * x                 (x) float64 -6.703e+05 -6.703e+05 ... -6.626e+05
Data variables:
    blue_s2_gm        (y, x) float32 674.1 678.6 834.2 ... 287.8 335.0 355.8
    green_s2_gm       (y, x) float32 995.8 988.7 1.203e+03 ... 469.0 576.0 697.9
    red_s2_gm         (y, x) float32 1.285e+03 1.408e+03 ... 452.7 518.0
    nir_s2_gm         (y, x) float32 2.513e+03 2.241e+03 ... 3.472e+03 4.093e+03
    swir_1_s2_gm      (y, x) float32 3.004e+03 3.003e+03 ... 1.432e+03 1.692e+03
    swir_2_s2_gm      (y, x) float32 2.45e+03 2.455e+03 ... 743.3 884.4
    red_edge_1_s2_gm  (y, x) float32 1.839e+03 1.834e+03 ... 821.0 1.049e+03
    red_edge_2_s2_gm  (y, x) float32 2.248e+03 2.241e+03 ... 2.383e+03 3.401e+03
    red_edge_3_s2_gm  (y, x) float32 2.428e+03 2.422e+03 ... 2.913e+03 4.056e+03
    NDVI_s2_gm        (y, x) float32 0.3233 0.2281 0.1535 ... 0.7693 0.7753
    SAVI_s2_gm        (y, x) float32 0.2094 0.1443 0.1039 ... 0.5075 0.558
    NDMI_s2_gm        (y, x) float32 -0.08895 -0.1453 -0.1413 ... 0.4159 0.4151

In [9]:
im_layer = mk_image_overlay(ds, clamp=3000 ,bands=['red_s2_gm', 'green_s2_gm', 'blue_s2_gm'], fmt='png')
im = Image.fromarray(im_layer[1].values)
im.save("delineation_aoi.png")

In [10]:
rgb_im = cv2.imread('delineation_aoi.png')
enhanced_img = enhance_image(img = im_layer[1].values, contrast=4, sharpness=8, brightness=3)
equalized_img = equalize_histogram(enhanced_img)
edges_ = find_edges(equalized_img)
structure_edges_ = structure_edges(edges_)
contours_ = find_contours(structure_edges_, rgb_im)
approximations_ = approximate_contours(contours_)
polygons_ = extract_polygons(approximations_)

In [11]:
inProj = Proj(init='epsg:6933')
outProj = Proj(init='epsg:4326')
convert_lists = []
for j in range(len(polygons_)):
    convert_lists.append(convert_list(polygons_[j]))

In [13]:
d = {'col1': list(range(len(polygons_))), 'geometry': [Polygon(converted_list) for converted_list in convert_lists] }
farm_plots_gdf = gpd.GeoDataFrame(d).set_crs('epsg:4326')

In [15]:
m = leafmap.Map(basemap=basemaps.Esri.WorldImagery)
m.add_gdf(farm_plots_gdf[2:], layer_name="detected farm plots", fill_colors=['red'])
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…